# Homework 1 Due Date -8th Sep (11:59 pm)


Submissions: Please submit .ipynb file and pdf file to eLearning

Megan Yiu - mny170000

In [ ]:
import torch
import numpy as np
import time

## Question 1. Speedtest for vectorization - 1 Point

Your goal is to measure the speed of linear algebra operations for different levels of vectorization.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $1024 \times 1024$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. (Hint: Use torch.mm)
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time. (hint use torch.mv inside a for loop)
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time. (Hint: use torch.dot inside nested for loop)

In [ ]:
## Solution 1
torch.manual_seed(42)
# your code here
A= torch.randn(1024, 1024)
B= torch.randn(1024, 1024)

In [ ]:
## Solution 2
start=time.time()

# your code here
torch.mm(A, B)
print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.07364797592163086 seconds


In [ ]:
## Solution 3
C= torch.empty(1024,1024)
start = time.time()
iterations = 0;

# your code here
for col in range(B.shape[1]):
  C[:,col] = torch.mv(A, B[:,col])

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.2188282012939453 seconds


In [ ]:
## Solution 4
C= torch.empty(1024,1024)
start = time.time()

# your code here
for row in range(A.shape[0]):
  for col in range(B.shape[1]):
      C[row, col] = torch.dot(A[row,:],B[:,col])

print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 20.376437425613403 seconds


## Question 2 : Redo Question 1 by using GPU - 1 Point

##Using GPUs

How to use GPUs in Google Colab<br>
In Google Colab -- Go to Runtime Tab at top -- select change runtime type -- for hardware accelartor choose GPU

In [ ]:
import torch
import numpy as np
import time

In [ ]:
# Check if GPU is availaible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((1024, 1024),device=device)
B= torch.randn((1024, 1024),device=device)

#check device of each tensor
print(A.device)
print(B.device)

cuda:0
cuda:0


In [ ]:
## Solution 2
start=time.time()

# your code here
C = torch.mm(A,B)
print("Matrix by matrix: " + str(time.time()-start) + " seconds")
#print(C)

Matrix by matrix: 0.009042024612426758 seconds


In [ ]:
## Solution 3
C= torch.empty((1024, 1024),device=device)
start = time.time()

# your code here
for col in range(B.shape[1]):
  column = B[:,col]
  C[:,col] = torch.mv(A, column)

print("Matrix by vector: " + str(time.time()-start) + " seconds")  

Matrix by vector: 0.048941612243652344 seconds


In [ ]:
## Solution 4
D= torch.empty((1024, 1024),device=device)
start = time.time()

# your code here
for row in range(A.shape[0]):
  for col in range(B.shape[1]):
      D[row, col] = torch.dot(A[row,],B[:,col])

print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 56.00712466239929 seconds


## Question 3. PyTorch Tensor and NumPy - 1.5 Points

Your goal is to measure the speed penalty between PyTorch Tensors and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $10000 \times 10000$ in PyTorch. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{10000}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

<br>**Hint : each elemnt of $c_i$ can be computed as 
   

```
torch.norm(torch.mv(A,B[:,i]))**2
```

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in PyTorch for assignments and copy to NumPy at the end.

In [ ]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((10000, 10000),device=device)
B= torch.randn((10000,10000),device=device)

In [ ]:
## Solution 2a - One at a time
c=np.empty(10000)
start=time.time()
for i in range(10000):
  
  c[i] = torch.norm(torch.mv(A,B[:,i]))**2

print(" one at a time: " + str(time.time()-start)+ " seconds")

 one at a time: 38.762749433517456 seconds


In [ ]:
## Solution 2b - Copy to numpy at the end
d= torch.empty(10000,device=device)
start= time.time()
for i in range(10000):
  
  d[i]= torch.norm(torch.mv(A,B[:,i]))**2

c= d.cpu().numpy()
print("convert at the end: " + str(time.time()-start)+ "seconds")

convert at the end: 38.020729541778564seconds


## Question 4. Memory efficient computation - 1.5 Point

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [ ]:
A= torch.randn((1000, 1000),device=device)
B= torch.randn((1000,1000),device=device)
C= torch.randn((1000, 1000),device=device)
id(C)

139687624700784

In [ ]:
# your code here
C += torch.mm(A, B)
id(C)

139687624700784

## Question 5. Broadcast Operations - 1.5 Point

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 3$ and $x = \{1,2,3,4,5\}$. Implement code that generates following A matrix

$$\begin{bmatrix} 1 & 1 & 1 \\ 2 & 4 & 8 \\ 3 & 9 & 27 \\ 4 & 16 & 64 \\ 5 & 25 & 125\end{bmatrix}$$

In [ ]:
# your code here
x = torch.arange(1, 6).reshape(5,1);
j = torch.arange(1, 4);
print(x ** j)

tensor([[  1,   1,   1],
        [  2,   4,   8],
        [  3,   9,  27],
        [  4,  16,  64],
        [  5,  25, 125]])


## Question 6. Numerical Precision - 2 Points

Given scalars `x` and `y`, implement the following `log_exp` function such that it returns 
$$-\log\left(\frac{e^x}{e^x+e^y}\right)$$.

In [ ]:
#Question
def log_exp(x, y):
    ## add your solution here and remove pass
   return -torch.log((torch.exp(x)) / (torch.exp(x) + torch.exp(y)))
   

Test your codes with normal inputs:

In [ ]:
x, y = torch.tensor([2.0]), torch.tensor([3.0])
z = log_exp(x, y)
print(z)


tensor([1.3133])


Now implement a function to compute $\partial z/\partial x$ and $\partial z/\partial y$ with `autograd`

In [ ]:
# function should print the gradients dx and dy 
def grad(forward_func, x, y): 
  x.requires_grad_()
  y.requires_grad_()
  ## Add your codes here
  #forward pass and backward pass
  f = forward_func(x,y)
  f.backward()

  #return and print x and y gradients
  print(x.grad)
  print(y.grad)


Test your codes, it should print the results nicely. 

In [ ]:
grad(log_exp, x, y)

tensor([-0.7311])
tensor([0.7311])


But now let's try some "hard" inputs

In [ ]:
x, y = torch.tensor([50.0]), torch.tensor([100.0])

In [ ]:
grad(log_exp, x, y) #gradient does not return correct results

tensor([nan])
tensor([nan])


In [ ]:
torch.exp(torch.tensor([100.0])) #exp(100) returns an infinite tensor/overflow so above gradient does not return correct results

tensor([inf])

Does your code return correct results? If not, try to understand the reason. (Hint, evaluate `exp(100)`). Now develop a new function `stable_log_exp` that is identical to `log_exp` in math, but returns a more numerical stable result.
<br> Hint: (1) $\log\left(\frac{x}{y}\right) = log ({x}) -log({y})$
<br> Hint: (2) See logsum Trick - https://www.xarg.org/2016/06/the-log-sum-exp-trick-in-machine-learning/

In [ ]:
def stable_log_exp(x, y):
    ## Add your codes here)
    # -log(exp(x)) - -log(exp(x) + exp(y)) -> log(exp(x) + exp(y) )
    return torch.logaddexp(x,y)  - torch.logsumexp(x, -1)

In [ ]:
grad(stable_log_exp, x, y)

tensor([-1.])
tensor([1.])


##  Question 7. Compute Gradient using  PyTorch Autograd - 1.5 Points
## $f(x,y) = \frac{x + \exp(y)}{\log(x) + (x-y)^3}$
Compute dx and dy at x=3 and y=4

In [ ]:
# your code here
x = torch.tensor([3.])
y = torch.tensor([4.])

x.requires_grad_()
y.requires_grad_()

forward = (x + torch.exp(y)) / (torch.log(x) + (x-y)**3)
forward.backward()

dx = x.grad
dy = y.grad

print(dx)
print(dy)




tensor([-19733.3965])
tensor([18322.8477])


Let us check our solution using PyToch autodiff

## Bonus Question : Calculate the gradients manually without using PyTorch's Autograd. 
- Break the function into simpler functions. Calculate the forward pass
- Use backward pass (chain rule) to calculate the gradients.

In [ ]:
#forward function breaking down the original function into smaller functions
a = None
b = None

def f_intermediate(x, y):
  global a, b
  a = x + torch.exp(y)
  a.retain_grad()
  b = torch.log(x) + (x-y)**3
  b.retain_grad()

  #final function
  z = a / b
  z.retain_grad()
  return z

with torch.no_grad():
  dea = b 
  deb = a 

  dax = torch.FloatTensor([1.0])
  dby = -3*(x-y)**2
  dbx = 3*(x-y)**2 + (1/x)



tensor([584.0868], grad_fn=<DivBackward0>)